In [6]:
import torch
from torch.utils.data.distributed import DistributedSampler
import os
# from .mel2samp import Mel2Samp

class Mel2Samp(torch.utils.data.Dataset):
    """
    This is the main class that calculates the spectrogram and returns the
    spectrogram, audio pair.
    """
    def __init__(self, data_path):
        # self.file_list = os.listdir(data_path)#文件类型：org；scaled
        self.data_path = data_path
        self.label_list = []
        self.file_list = []
        for label in os.listdir(data_path): 
            for filename in os.listdir(os.path.join(data_path, label)):
               self.file_list.append(filename) 
               self.label_list.append(label)
            
    def __getitem__(self, index):
        # Read audio 
        filename = self.file_list[index]
        label = self.label_list[index]
        
        file = torch.load(os.path.join(self.data_path, label, filename))
        
        return (file, int(label))

    def __len__(self):
        return len(self.file_list)
    
    
def load_LJSpeech(data_path, batch_size=16, num_gpus=1):
    LJSpeech_dataset = Mel2Samp( data_path= data_path)

    # distributed sampler
    train_sampler = DistributedSampler(LJSpeech_dataset) if num_gpus > 1 else None

    trainloader = torch.utils.data.DataLoader(LJSpeech_dataset, 
                                              batch_size=batch_size,  
                                              sampler=train_sampler,
                                              num_workers=4,
                                              pin_memory=False,
                                              drop_last=True)
    return trainloader


In [7]:
LJSpeech_dataset = Mel2Samp( data_path= 'E:/datasets/project3/LJSpeech-1.1/train/classifier')

In [3]:
L = load_LJSpeech('E:/datasets/project3/LJSpeech-1.1/train/classifier')

In [4]:
len(L)

6547

In [8]:
 x, y = next(iter(LJSpeech_dataset))

In [9]:
x.shape

torch.Size([80, 164])

In [10]:
y 

-1

In [ ]:
for i,ele in L:
    for i in range(10):
        print(ele[0].shape)